### Лабораторная работа № 5 ###
#### Линейные модели, SVM и деревья решений ####
**Цель лабораторной работы:** изучение линейных моделей, SVM и деревьев решений.

Выполнил: Мокренко Никита ИУ5-64

1. Выберите набор данных (датасет) для решения задачи классификации или регресии.
2. В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.
3. С использованием метода train_test_split разделите выборку на обучающую и тестовую.
4. Обучите 1) одну из линейных моделей, 2) SVM и 3) дерево решений. Оцените качество моделей с помощью трех подходящих для задачи метрик. Сравните качество полученных моделей.
5. Произведите для каждой модели подбор одного гиперпараметра с использованием GridSearchCV и кросс-валидации.
6. Повторите пункт 4 для найденных оптимальных значений гиперпараметров. Сравните качество полученных моделей с качеством моделей, полученных в пункте 4.

In [60]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import *
from sklearn.datasets import load_iris

### 1. Выберем набор данных

In [61]:
df = load_iris()
df = pd.DataFrame(data = np.c_[df['data'], df['target']], columns = df['feature_names'] + ['target'])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


Пропусков нет

In [62]:
df.dtypes

sepal length (cm)    float64
sepal width (cm)     float64
petal length (cm)    float64
petal width (cm)     float64
target               float64
dtype: object

Категориальных признаков нет

In [63]:
df.isnull().sum()

sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
target               0
dtype: int64

### 3. Деление выборки на обучающую и тестовую

In [64]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    df.loc[:, df.columns != 'target'], df['target'],
    test_size=0.2, random_state=1) 

### 4.1. Линейная регрессия

In [65]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(x_train, y_train)
target1 = lr.predict(x_test)
lr.intercept_

0.39942667627255557

In [66]:
print('Средняя абсолютная ошибка: ', mean_absolute_error(y_test, target1), '\n', 
      'Коэффициент детерминации: ',r2_score(y_test, target1), sep = '')

Средняя абсолютная ошибка: 0.20421913597428992
Коэффициент детерминации: 0.8843648990647469


### 4.2. Метод опорных векторов

In [67]:
from sklearn.svm import SVR

svr = SVR()
svr.fit(x_train, y_train)
target2 = svr.predict(x_test)

/usr/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [68]:
print('Средняя абсолютная ошибка: ', mean_absolute_error(y_test, target2), '\n', 
      'Коэффициент детерминации: ',r2_score(y_test, target2), sep = '')

Средняя абсолютная ошибка: 0.1245766430879443
Коэффициент детерминации: 0.9433799561982495


### 4.3. Дерево решений

In [69]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor()
dtr.fit(x_train, y_train)
target3 = dtr.predict(x_test)

In [70]:
print('Средняя абсолютная ошибка: ', mean_absolute_error(y_test, target3), '\n', 
      'Коэффициент детерминации: ',r2_score(y_test, target3), sep = '')

Средняя абсолютная ошибка: 0.03333333333333333
Коэффициент детерминации: 0.9381443298969072


### 5.1. Подбор параметров для линейной регрессии

In [71]:
from sklearn.model_selection import GridSearchCV

l_range = np.array(range(1,10))

linear_parameters = [{'n_jobs': l_range}]

lr_gs = GridSearchCV(LinearRegression(), linear_parameters, cv = 3, scoring = 'r2')
lr_gs.fit(x_train, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_jobs': array([1, 2, 3, 4, 5, 6, 7, 8, 9])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [72]:
print('Лучшая модель: ', lr_gs.best_estimator_, '\n', 'Лучшее значение метрики: ', lr_gs.best_score_,
      '\n', 'Лучшее значение параметров: ', lr_gs.best_params_, sep = '')

Лучшая модель: LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
Лучшее значение метрики: 0.9274227354928587
Лучшее значение параметров: {'n_jobs': 1}


### 5.2. Подбор параметров для метода опорных векторов

In [73]:
v_range = np.array(range(1,10))

vector_parameters = [{'degree': v_range}]

lr_gs = GridSearchCV(SVR(), vector_parameters, cv = 3, scoring = 'r2')
lr_gs.fit(x_train, y_train)

/usr/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will chan

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'degree': array([1, 2, 3, 4, 5, 6, 7, 8, 9])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [74]:
print('Лучшая модель: ', lr_gs.best_estimator_, '\n', 'Лучшее значение метрики: ', lr_gs.best_score_,
      '\n', 'Лучшее значение параметров: ', lr_gs.best_params_, sep = '')

Лучшая модель: SVR(C=1.0, cache_size=200, coef0=0.0, degree=1, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)
Лучшее значение метрики: 0.9459363563268082
Лучшее значение параметров: {'degree': 1}


### 5.3. Подбор параметров для дерева решений

In [75]:
t_range = np.array(range(1,30))

vector_parameters = [{'max_depth': t_range}]

lr_gs = GridSearchCV(DecisionTreeRegressor(), vector_parameters, cv = 3, scoring = 'r2')
lr_gs.fit(x_train, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [76]:
print('Лучшая модель: ', lr_gs.best_estimator_, '\n', 'Лучшее значение метрики: ', lr_gs.best_score_,
      '\n', 'Лучшее значение параметров: ', lr_gs.best_params_, sep = '')

Лучшая модель: DecisionTreeRegressor(criterion='mse', max_depth=13, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')
Лучшее значение метрики: 0.9374238744515823
Лучшее значение параметров: {'max_depth': 13}


### 6.1. Линейная регрессия

In [77]:
lr1 = LinearRegression(n_jobs = 1)
lr1.fit(x_train, y_train)
target1_1 = lr.predict(x_test)
lr1.intercept_

0.39942667627255557

In [78]:
print('Средняя абсолютная ошибка: ', mean_absolute_error(y_test, target1), '\n', 
      'Коэффициент детерминации: ',r2_score(y_test, target1_1), sep = '')

Средняя абсолютная ошибка: 0.20421913597428992
Коэффициент детерминации: 0.8843648990647469


### 6.2. Метод опорных векторов

In [79]:
svr1 = SVR(degree =1)
svr1.fit(x_train, y_train)
target2_1 = svr.predict(x_test)

/usr/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [80]:
print('Средняя абсолютная ошибка: ', mean_absolute_error(y_test, target2_1), '\n', 
      'Коэффициент детерминации: ',r2_score(y_test, target2_1), sep = '')

Средняя абсолютная ошибка: 0.1245766430879443
Коэффициент детерминации: 0.9433799561982495


### 6.3. Дерево решений

In [81]:
dtr1 = DecisionTreeRegressor(max_depth = 26)
dtr1.fit(x_test, y_test)
target3_1 = dtr1.predict(x_test)

In [83]:
print('Средняя абсолютная ошибка: ', mean_absolute_error(y_test, target3_1), '\n', 
      'Коэффициент детерминации: ',r2_score(y_test, target3_1), sep = '')

Средняя абсолютная ошибка: 0.0
Коэффициент детерминации: 1.0


![wow](https://thumbs.gfycat.com/PalePointedIrrawaddydolphin-size_restricted.gif "wow")